Human

In [2]:
import cv2
import numpy as np

In [3]:
cv2.__version__

'4.5.5'

In [4]:
# pip install opencv-contrib-python == 4.5.5

ERROR: Invalid requirement: '=='


In [ ]:
# lib.01gl c++ 
#opencv 4.9.8 (error)
#opencv-python-heddless
#The Kernel crashed

In [4]:
img = cv2.imread("./data/human.jpg")

In [5]:
type(img)

numpy.ndarray

In [6]:
copy = img.copy()

In [8]:
cv2.imshow("image", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [9]:
img.shape

(510, 768, 3)

In [10]:
#mask
mask = np.zeros(img.shape[:2], np.uint8)

In [11]:
#create dummay arrays for future using
bgd = np.zeros((1,65), np.float64)
fgd = np.zeros((1,65), np.float64)

In [12]:
#select ROI
rect = cv2.selectROI(img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [14]:
x,y,w,h = rect

In [ ]:
cv2.rectangle(copy, (x,y), (x+w, y+h), (0,0,255), 3)

In [16]:
cv2.imwrite("roi.png", copy)

True

In [ ]:
cv2.grabCut(img, mask, rect, bgd, fgd, 5, cv2.GC_INIT_WITH_RECT)

In [24]:
np.unique(mask)

array([0, 2, 3], dtype=uint8)

In [31]:
mask2 = np.where((mask == 2 ) | (mask == 0), 0, 1).astype("uint8")
cv2.imwrite("mask.png", mask*80)
cv2.imwrite("mask2.png", mask2*255)


True

In [29]:
np.unique(mask)

array([0, 2, 3], dtype=uint8)

In [30]:
np.unique(mask2)

array([0, 1], dtype=uint8)

In [34]:
mask.ndim,mask2.ndim

(2, 2)

In [39]:
#inference on original data (image)

img = img * mask2[:,:,np.newaxis]
cv2.imwrite("result.png", img)

True

Cars

In [51]:
haar_cascade_car = "./data/haarcascade_car.xml"

In [52]:
car_class = cv2.CascadeClassifier(haar_cascade_car)

In [53]:
#read video
cap = cv2.VideoCapture("./data/cars.avi")

In [ ]:
count = 0

In [54]:
# object detection (cars)

while cap.isOpened():

    count +=1

    if count % 20 ==0:

        ret, frame = cap.read()

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        cars = car_class.detectMultiScale(gray, 1.1, 3)

        for (x, y, w, h) in cars:

            cv2.rectangle(frame, (x,y), (x+w, y+h), (0,255,0), 3)

            cv2.putText(frame,"car",(x, y-10), cv2.FONT_HERSHEY_SCRIPT_SIMPLEX,0.5, (0,0,255))

        cv2.imshow("cars", frame)
    
    if cv2.waitKey(0) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()

Lemon

In [10]:
import cv2
import numpy as np

In [7]:
cv2.__version__

'4.5.5'

In [13]:
cap = cv2.VideoCapture("./data/lemon.mp4")

In [5]:
type(cap)

cv2.VideoCapture

In [ ]:
# RGB , LAB, HSV

In [5]:
cv2.namedWindow("input")
cv2.namedWindow("output")
cv2.namedWindow("Hue channel")
cv2.waitKey(0)
cv2.destroyAllWindows()

In [14]:
while True:

    ret, frame = cap.read()

    if ret == False:
        break

    #convert frames to HSV format
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    #select the hue channel (0) --> h 0 | s 1 | v 2
    h = hsv[:,:,0]

    #apply threshold
    copy = h.copy()
    h[copy>40] = 0
    h[copy<=40] = 1

    #show windows (images)
    cv2.imshow("input", frame)
    # cv2.imshow("output", frame * h[:,:,np.newaxis])
    # cv2.imshow("hue", h * 255)

    #close windows & waiting time
    if cv2.waitKey(0) & 0xFF == ord("q"):
        break


cap.release()
cv2.destroyAllWindows()


In [19]:
#help & guide

img = cv2.imread("./data/human.jpg")
img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

cv2.imshow("human",img[:,:,0])
cv2.waitKey(0)
cv2.destroyAllWindows()

People

In [1]:
import cv2
import numpy as np

In [2]:
video = cv2.VideoCapture("./data/people.mp4")

In [3]:
# checking if we opened video
if video.isOpened()== False:
    print("not opened")
else:
    print("opened")

opened


In [3]:
ret, frame = video.read()

In [5]:
type(ret), type(frame), frame.shape

(bool, numpy.ndarray, (360, 640, 3))

In [7]:
ret

True

In [9]:
# cv2.imshow("first", frame)
cv2.imwrite("firstframe.jpg", frame)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

True

In [4]:
# checking first frame
if ret == False:
    print("can not read frame")
else:
    print("read frame")

read frame


In [4]:
box = cv2.selectROI(frame)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [11]:
box

(52, 17, 167, 342)

In [5]:
# track the selected object using medianflow tracker
tracker = cv2.legacy.TrackerMedianFlow_create()

In [19]:
type(tracker)

cv2.legacy_TrackerMedianFlow

In [21]:
cv2.__version__

'4.5.5'

In [6]:
#warm-up out tracker
ret = tracker.init(frame, box)

In [24]:
cv2.imshow("tracker", frame)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [7]:
while True:

    #read all frames (next - next - next)
    ret, frame = video.read()

    #check frame
    if ret == False:
        break

    #update tracker
    found, bbox = tracker.update(frame)

    if found:

        #top-left corner
        topleft = (int(bbox[0]), int(bbox[1]))

        #bottom-right corner
        bottomright = (int(bbox[0]+bbox[2]), int(bbox[1]+bbox[3]))

        #display box
        cv2.rectangle(frame, topleft, bottomright,(0,0,255),3)
    else:
        #display status
        cv2.putText(frame, "not found", (20,70), cv2.FONT_HERSHEY_SCRIPT_SIMPLEX,0.75, (0,0,255),3)
        
    #display frame
    cv2.imshow("tracker", frame)
    space = cv2.waitKey(0)
    if space == 30:
        break
    


In [8]:
cv2.destroyAllWindows()   

The End